# Machine Translation

English-German Translation

# 1)- Importing key modules

In [1]:
#support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# What's life without style :). So, let's add style to our dataframes
#from IPython.core.display import HTML
#css = open('style-table.css').read() + open('style-notebook.css').read()
#HTML('<style>{}</style>'.format(css))

In [3]:
import pandas as pd 
import string 
import pickle
from pickle import dump
from pickle import load
from string import digits
import re 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from numpy import array, argmax, random, take 
import matplotlib.pyplot as plt 
%matplotlib inline 
pd.set_option('display.max_colwidth', 200)

In [4]:
from keras.models import Model
from keras.models import Sequential 
from keras.layers import Dense, LSTM, Embedding,Input,RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint 
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model 
from keras import optimizers 

Using TensorFlow backend.


In [5]:
#!  pip install version_information

In [6]:
# first install: pip install version_information
#%reload_ext version_information
#%version_information pandas,re,sklearn, matplotlib,keras

Software versions
Python 3.6.8 64bit [GCC 8.0.1 20180414 (experimental) [trunk revision 259383]
IPython 5.5.0
OS Linux 4.14.137+ x86_64 with Ubuntu 18.04 bionic
pandas 0.24.2
re 2.2.1
sklearn 0.21.3
matplotlib 3.0.3
keras 2.2.5
Tue Sep 10 15:29:44 2019 UTC

# 2)- Reading Dataset

In [5]:
lines= pd.read_pickle('full_data.pkl')

In [6]:
lines.shape

(22191, 2)

In [7]:
lines.head()

,eng,ger
0,Prague Stock Market falls to minus by the end of the trading day,Die Prager Börse stürzt gegen Geschäftsschluss ins Minus.
1,After a sharp drop in the morning,Nach dem steilen Abfall am Morgen konnte die Prager Börse die Verluste korrigieren.
2,Transactions with stocks from the Czech Energy Enterprise (ČEZ) reached nearly half of the regular daily trading.,Die Transaktionen mit den Aktien von ČEZ erreichten fast die Hälfte des normalen Tagesgeschäfts.
3,The Prague Stock Market immediately continued its fall from Monday at the beginning of Tuesday's trading,Die Prager Börse knüpfte gleich zu Beginn der Dienstagsgeschäfte an den Einbruch vom Montag an
4,This time the fall in stocks on Wall Street is responsible for the drop.,Diesmal lag der Grund für den Einbruch an der Wall Street.


As this is big data and I have a poor old computing machine. So, I ll use smaller sample. It got to be random to avoid sample biaseness

In [8]:
lines.sample(15)

,eng,ger
9439,"Asked by Sawyer how she feels, Giffords, 41, responded, ""Pretty good.""","Als Sawyer sie fragte, wie sie sich fühle, antwortete Gifford (41): ""Ganz gut."""
2241,Italians,"""deren Lohn-Dynamiken - so der Minister - waren in den letzten 8-10 Jahre doppelt so groß wie der Privatbereich und weit über der Inflationsrate""."
13901,Do you want him?',"Möchtest Du ihn?"""
22157,"Who is going to take the role of Mary Poppins, played 50 years ago by Julie Andrews, is not yet certain.","Wer Poppins mehr als 50 Jahre nach Julie Andrews spielt, ist noch unklar."
22167,"The government will also loosen controls on the powers it delegates to companies, making sure that companies decide how to run their businesses.","Die Regierung wird auch Kontrollen über die an die Unternehmen übertragenen Befugnisse lockern und dafür sorgen, dass Unternehmen entscheiden, wie sie ihre Geschäfte führen."
705,Hamburg has to thank Mladen Petric for scoring the 1-0 (1-0) with his header (11th); and the fact that a very weak Gladbach team proved unable to hit the weak Hamburg team where it hurts.,Das 1:0 (1:0) haben die Hamburger dem Kopfballtreffer von Mladen Petric zu verdanken (11.); und dem Umstand
7739,NIcolas Sarkozy receives her at the Elysée.,Nicolas Sarkozy empfängt sie im Elysée.
106,It's enough to fill the device with music using the USB from the computer.,Sie brauchen das Gerät nur über USB-Port mit Musik aus Ihrem PC zu füttern.
21662,Friendship lies at the heart of the battle to meet this challenge.,"Freundschaft ist das Herzstück im Kampf, um diese Herausforderung zu meistern."
10033,"Casillas and Xavi were recognized last week by UEFA together with the other player of the Iberian team, Zubizarreta with 126 matches and Schalke striker Raul Gonzalez with 102.","Casillas und Xavi erhielten letzte Woche sogar eine Anerkennung von der UEFA, zusammen mit anderen der iberischen Mannschaft, die die magische Zahl 100 erreicht hatten: der zitierte Zubizarreta mi..."


In [9]:
lines = lines.sample(2000)

# 3)- Quick Preprocessing

In [10]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.deu=lines.ger.apply(lambda x: x.lower())

In [11]:
# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.deu=lines.ger.apply(lambda x: re.sub("'", '', x))

In [12]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.deu=lines.ger.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [13]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.deu=lines.ger.apply(lambda x: x.translate(remove_digits))

In [14]:
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.deu=lines.ger.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.deu=lines.ger.apply(lambda x: re.sub(" +", " ", x))

In [15]:
lines.sample(10)

,eng,ger
465,ultimately,Am Ende könnte auch eine Zerlegung der West LB stehen
16994,the national intelligence director james clapper defended spying on allies as necessary and said its commonplace on both sides,"Der Leiter der nationalen Geheimdienste, James Clapper, verteidigte die Spionage bei Verbündeten als notwendig und erklärte, dies sei auf beiden Seiten üblich."
5474,later he very conveniently exchanged those with the city for a house in the historic town center which had a much higher value,"Später tauschte er diese sehr günstig mit dem Magistrat gegen ein Haus direkt im historischen Zentrum der Stadt ein, das den Wert der Grundstücke weit überstieg."
5689,the threeday foundation course for future gelatieri is the largest course and welcomes visitors from all over the world,Der dreitägige Grundkurs für künftige Gelatieri hat den größten Zulauf und Besucher aus der ganzen Welt.
2877,back when i renounced my mandate i knew that the following months would see the bolshevisation of the house and the destruction of all that is positive topolánek himself responded,"Schon als ich auf das Mandat verzichtete, wußte ich, dass es in nächsten Monaten vor allem um Bolschewisierung des Parlaments und Zerstörung alles Positiven gehen würde, reagierte Topolanek selbst."
10253,the ministry of communications and transportation mct reported that the results obtained on the accident in which the interior secretary francisco blake mora and seven others died indicate that th...,"Die Secretaría de Comunicaciones y Transportes - SCT (Sekretariat für Kommunikation und Transport in Mexiko) informierte, dass die Untersuchungsergebnisse über den Unfall, bei dem der Regierungsse..."
18868,the ninth edition of calculus by ron larson bruce edwards and robert hostetler carries a list price of nearly but can be purchased new for at specialty textbook retailer cheggcom,"Die neunte Ausgabe von ""Calculus"" vno Ron Larwon, Bruce Edwards und Robert Hostetler kostet fast 290 $, kann jedoch bei dem spezialisierten Lehrbucheinzelhändler Chegg.com für 239,99 $ bezogen wer..."
13544,garavanis life is not a story of obsession but of well reciprocated love,"Das Leben von Garavani ist keine Geschichte von Versessenheit, eher von gegenseitigen Liebschaften."
4629,and equally different is the hair style on this occasion hair tied back previously she wore her hair down,"Auch die Frisur war jedes Mal anders: Die Haare waren bei diesem Anlass als Haarkranz hochgesteckt, beim vorherigen trug sie die Haare offen."
404,jaroslav hořín,Der BESIP-Fachmann für Verkehrssicherheit Jaroslav Hořín sieht in der Bewegung von Stewards an Bord eines Busses kein grundlegendes Problem.


In [16]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
# Vocabulary of German 
all_german_words=set()
for ger in lines.ger:
    for word in ger.split():
        if word not in all_german_words:
            all_german_words.add(word)

In [17]:
# Max Length of source sequence
import numpy as np
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

86

In [18]:
# Max Length of target sequence
lenght_list=[]
for l in lines.ger:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

86

# Make a threshold here

### 3a)- Defining input and target

In [19]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_german_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_german_words)
num_encoder_tokens, num_decoder_tokens

(8262, 13119)

In [20]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

13120

In [21]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [22]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

### 3b)-Train - Test Split

For validation

In [23]:
X, y = lines.eng, lines.ger #X being input, y being target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train.shape, X_test.shape

((1600,), (400,))

**Save the train and test dataframes for reproducing the results later, as they are shuffled**

In [24]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [25]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

# 4)-Encoder - Decoder Model Architecture

In [26]:
latent_dim = 50

In [27]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [28]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [29]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [30]:
#from IPython.display import Image
#Image(retina=True, filename='train.png')

In [31]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 15

In [32]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     413100      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     656000      input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [33]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/15
 5/12 [===========>..................] - ETA: 1:06 - loss: 9.4768 - acc: 0.0104  

InvalidArgumentError: indices[42,3] = 8262 is not in [0, 8262)
	 [[{{node embedding_1/embedding_lookup}}]]

In [0]:
# save model
model.save_weights('translate.h5')

In [0]:
model.load_weights('translate.h5')

# Inference Setup

In [0]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Decode sample sequeces

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

# Evaluation on Train Dataset

In [0]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [42]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual German Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted German Translation:', decoded_sentence[:-4])

KeyError: ignored

In [43]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual German Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted German Translation:', decoded_sentence[:-4])

KeyError: ignored

In [0]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual German Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted German Translation:', decoded_sentence[:-4])

# Evaluation on Validation Dataset

In [0]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [45]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual German Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted German Translation:', decoded_sentence[:-4])

KeyError: ignored

In [0]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual German Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted German Translation:', decoded_sentence[:-4])

In [0]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual German Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted German Translation:', decoded_sentence[:-4])